In [14]:
import os
import pandas as pd
from tqdm import tqdm

In [ ]:
class Seq:
    
    def __init__(self, name, description, sequence):
        self.name = name.lstrip('>')
        self.desc = description.split('(')[1].split(')')[0]
        self.seq = sequence.lower()


def read_seq(file):
    with open(file, 'r') as f:
        con = [i.rstrip('\n') for i in f.readlines()]
    return Seq(con[0], con[1], ''.join(con[2:]))


def load_sequences(folder):
    seq_files = [os.path.join(folder, i) for i in os.listdir(folder) if not i.startswith('.')]
    seqs = map(read_seq, seq_files)
    seqs_df_rows = [(x.name, x.desc, x.seq) for x in tqdm(seqs)]
    seqs_df = pd.DataFrame(seqs_df_rows, columns=['Name','CDSjoin','Sequence'])
    return seqs_df

In [24]:
class WAM:
    
    def __init__(self, ):
        self.weights = pd.DataFrame()


In [25]:
class BN:
    def __init__(self, ):
        pass
        

In [26]:
class SVM:
    def __init__(self, ):
        pass
        

In [29]:
class Model:
    
    def __init__(self):
        self.WAM = WAM()
        self.BN = BN()
        self.SVM = SVM()